Tokenizar - Unicode y evaluación de librerías

In [1]:
# Instalamos las librerías necesarias (si no están instaladas)
!pip install PyMuPDF spacy nltk stanza transformers

  Obtaining dependency information for PyMuPDF from https://files.pythonhosted.org/packages/4a/26/8c72973b8833a72785cedc3981eb59b8ac7075942718bbb7b69b352cdde4/pymupdf-1.26.3-cp39-abi3-win_amd64.whl.metadata
  Obtaining dependency information for spacy from https://files.pythonhosted.org/packages/92/e7/8176484801c67dcd814f141991fe0a3c9b5b4a3583ea30c2062e93d1aa6b/spacy-3.8.7-cp311-cp311-win_amd64.whl.metadata
  Obtaining dependency information for nltk from https://files.pythonhosted.org/packages/4d/66/7d9e26593edda06e8cb531874633f7c2372279c3b0f46235539fe546df8b/nltk-3.9.1-py3-none-any.whl.metadata
  Obtaining dependency information for stanza from https://files.pythonhosted.org/packages/ac/f6/c50d0ee85e40687a81a95d90d426938d0d83ac0683efa2b87fa4110b665c/stanza-1.10.1-py3-none-any.whl.metadata
  Obtaining dependency information for transformers from https://files.pythonhosted.org/packages/db/5a/022ac010bedfb5119734cf9d743cf1d830cb4c604f53bb1552216f4344dc/transformers-4.55.2-py3-none-any.w


[notice] A new release of pip is available: 23.2.1 -> 25.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
!python -m spacy download es_core_news_sm

     ---------------------------------------- 0.0/12.9 MB ? eta -:--:--
     --------------------------------------- 0.0/12.9 MB 330.3 kB/s eta 0:00:39
     --------------------------------------- 0.1/12.9 MB 525.1 kB/s eta 0:00:25
     --------------------------------------- 0.1/12.9 MB 798.9 kB/s eta 0:00:16
      --------------------------------------- 0.3/12.9 MB 1.6 MB/s eta 0:00:08
     - -------------------------------------- 0.5/12.9 MB 2.2 MB/s eta 0:00:06
     -- ------------------------------------- 0.6/12.9 MB 2.5 MB/s eta 0:00:05
     -- ------------------------------------- 0.8/12.9 MB 2.5 MB/s eta 0:00:05
     -- ------------------------------------- 1.0/12.9 MB 2.7 MB/s eta 0:00:05
     --- ------------------------------------ 1.1/12.9 MB 2.7 MB/s eta 0:00:05
     --- ------------------------------------ 1.2/12.9 MB 2.8 MB/s eta 0:00:05
     --- ------------------------------------ 1.2/12.9 MB 2.6 MB/s eta 0:00:05
     ---- ----------------------------------- 1.3/12.9 M


[notice] A new release of pip is available: 23.2.1 -> 25.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [1]:
import fitz  # PyMuPDF para leer PDF
import unicodedata
import spacy
import nltk
import stanza
from transformers import AutoTokenizer

c:\Users\guill\Documents\GitHub\nlp-konrad\env\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# Descargar modelos necesarios
nltk.download('punkt')
stanza.download('es')
nlp_stanza = stanza.Pipeline('es')
nlp_spacy = spacy.load('es_core_news_sm')
tokenizer_bert = AutoTokenizer.from_pretrained("dccuchile/bert-base-spanish-wwm-cased")

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\guill\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
2025-08-13 23:48:13 INFO: Downloaded file to C:\Users\guill\stanza_resources\resources.json
2025-08-13 23:48:13 INFO: Downloading default packages for language: es (Spanish) ...
2025-08-13 23:48:14 INFO: File exists: C:\Users\guill\stanza_resources\es\default.zip
2025-08-13 23:48:17 INFO: Finished downloading models and saved to C:\Users\guill\stanza_resources
2025-08-13 23:48:17 INFO: Checking for updates to resources.json in case models have been updated.  Note: this behavior can be turned off with download_method=None or download_method=DownloadMethod.REUSE_RESOURCES
2025-08-13 23:48:17 INFO: Downloaded file to C:\Users\guill\stanza_resources\resources.json
2025-08-13 23:48:18 INFO: Loading these models for language: es (Spanish):
| Processor    | Package           |
------------------------------------
| tokenize     | combined     

In [4]:
# 1. Leer el archivo PDF
def leer_pdf(ruta):
    with fitz.open(ruta) as doc:
        texto = "\n".join([page.get_text("text") for page in doc])
    return texto

ruta_pdf = "texts/texto 1.1.pdf"
texto = leer_pdf(ruta_pdf)
print("Texto original:\n", texto[:500])

Texto original:
 El lenguaje es una de las herramientas centrales en nuestra vida social y profesional. 
Entre otras cosas, actúa como un medio para transmitir ideas, información, opiniones 
y sentimientos; así como para persuadir, pedir información, o dar ordenes. Asimismo, 
el lenguaje humano es algo que esta en constante cambio y evolución; y que puede 
llegar a ser muy ambiguo y variable. Tomemos por ejemplo la frase "comí una pizza con 
amigos" comparada con "comí una pizza con aceitunas"; su estructura es 


In [5]:
# 2. Convertir a minúsculas
texto_minusculas = texto.lower()
print("\nTexto en minúsculas:\n", texto_minusculas[:500])


Texto en minúsculas:
 el lenguaje es una de las herramientas centrales en nuestra vida social y profesional. 
entre otras cosas, actúa como un medio para transmitir ideas, información, opiniones 
y sentimientos; así como para persuadir, pedir información, o dar ordenes. asimismo, 
el lenguaje humano es algo que esta en constante cambio y evolución; y que puede 
llegar a ser muy ambiguo y variable. tomemos por ejemplo la frase "comí una pizza con 
amigos" comparada con "comí una pizza con aceitunas"; su estructura es 


In [6]:
# 3. Normalización Unicode (NFC)
def normalizar_unicode(texto):
    return unicodedata.normalize('NFC', texto)

texto_normalizado = normalizar_unicode(texto_minusculas)
print("\nTexto normalizado:\n", texto_normalizado[:500])


Texto normalizado:
 el lenguaje es una de las herramientas centrales en nuestra vida social y profesional. 
entre otras cosas, actúa como un medio para transmitir ideas, información, opiniones 
y sentimientos; así como para persuadir, pedir información, o dar ordenes. asimismo, 
el lenguaje humano es algo que esta en constante cambio y evolución; y que puede 
llegar a ser muy ambiguo y variable. tomemos por ejemplo la frase "comí una pizza con 
amigos" comparada con "comí una pizza con aceitunas"; su estructura es 


In [7]:
# Download the necessary data for Spanish tokenization
nltk.download('punkt')
nltk.download('spanish_grammars')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\guill\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package spanish_grammars to
[nltk_data]     C:\Users\guill\AppData\Roaming\nltk_data...
[nltk_data]   Package spanish_grammars is already up-to-date!


True

In [8]:
# 4. Tokenización con diferentes librerías

def tokenizar_spacy(texto):
    return [token.text for token in nlp_spacy(texto)]

def tokenizar_nltk(texto):
    return nltk.word_tokenize(texto, language='spanish')

def tokenizar_stanza(texto):
    doc = nlp_stanza(texto)
    return [word.text for sent in doc.sentences for word in sent.words]

def tokenizar_bert(texto):
    return tokenizer_bert.tokenize(texto)

In [9]:
# Aplicamos cada método
tokens_spacy = tokenizar_spacy(texto_normalizado)
tokens_nltk = tokenizar_nltk(texto_normalizado)
tokens_stanza = tokenizar_stanza(texto_normalizado)
tokens_bert = tokenizar_bert(texto_normalizado)

Token indices sequence length is longer than the specified maximum sequence length for this model (1945 > 512). Running this sequence through the model will result in indexing errors


In [10]:
# Mostramos los primeros 20 tokens de cada método
print("\nTokens con SpaCy:", tokens_spacy[:20])
print("\nTokens con NLTK:", tokens_nltk[:20])
print("\nTokens con Stanza:", tokens_stanza[:20])
print("\nTokens con BERT:", tokens_bert[:20])


Tokens con SpaCy: ['el', 'lenguaje', 'es', 'una', 'de', 'las', 'herramientas', 'centrales', 'en', 'nuestra', 'vida', 'social', 'y', 'profesional', '.', '\n', 'entre', 'otras', 'cosas', ',']

Tokens con NLTK: ['el', 'lenguaje', 'es', 'una', 'de', 'las', 'herramientas', 'centrales', 'en', 'nuestra', 'vida', 'social', 'y', 'profesional', '.', 'entre', 'otras', 'cosas', ',', 'actúa']

Tokens con Stanza: ['el', 'lenguaje', 'es', 'una', 'de', 'las', 'herramientas', 'centrales', 'en', 'nuestra', 'vida', 'social', 'y', 'profesional', '.', 'entre', 'otras', 'cosas', ',', 'actúa']

Tokens con BERT: ['el', 'lenguaje', 'es', 'una', 'de', 'las', 'herramientas', 'centrales', 'en', 'nuestra', 'vida', 'social', 'y', 'profesional', '.', 'entre', 'otras', 'cosas', ',', 'actúa']


**Mejorar Bert**

In [11]:
tokens_bert = tokenizer_bert.tokenize(texto_normalizado)

# Dividimos en fragmentos de 512 tokens
max_length = 512
chunks = [tokens_bert[i:i+max_length] for i in range(0, len(tokens_bert), max_length)]

print(f"Se generaron {len(chunks)} fragmentos de 512 tokens")

Se generaron 4 fragmentos de 512 tokens


In [12]:
def tokenizar_bert(texto):
    return tokenizer_bert.tokenize(texto)

In [13]:
print("\nTokens con BERT:", tokens_bert[:20])


Tokens con BERT: ['el', 'lenguaje', 'es', 'una', 'de', 'las', 'herramientas', 'centrales', 'en', 'nuestra', 'vida', 'social', 'y', 'profesional', '.', 'entre', 'otras', 'cosas', ',', 'actúa']
